In [ ]:
#Task 4 is about the sign langugaes detection

In [1]:
!pip install "numpy<2" --force-reinstall


  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [3]:
!pip install numpy==1.26.4


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tkinter import Tk, filedialog


def load_dataset(path, image_size=(64, 64)):
    X, y, labels = [], [], []
    for i, folder in enumerate(sorted(os.listdir(path))):
        fpath = os.path.join(path, folder)
        if not os.path.isdir(fpath):
            continue
        labels.append(folder)
        for file in os.listdir(fpath):
            img_path = os.path.join(fpath, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, image_size)
            X.append(img.flatten())
            y.append(i)
    return np.array(X), np.array(y), labels

dataset_path = r"C:\Users\HP\Downloads\archive (2)"  
X, y, labels = load_dataset(dataset_path)

print(f"Classes found: {labels}")
print(f"Total images: {len(X)}")

if len(X) == 0:
    raise ValueError("No images found! Check dataset path and folder structure.")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

print("Training complete.")
print(f"Test Accuracy: {model.score(X_test, y_test):.2f}")

#GUI
def predict_image():
    root = Tk()
    root.withdraw()  
    file_path = filedialog.askopenfilename(title="Select an image")
    if not file_path:
        print("No file selected.")
        return

    img = cv2.imread(file_path)
    if img is None:
        print("Invalid image.")
        return
    img_resized = cv2.resize(img, (64, 64)).flatten().reshape(1, -1)

    pred = model.predict(img_resized)[0]
    print(f"Prediction: {labels[pred]}")


predict_image()


Classes found: ['hurts', 'iloveyou', 'pain', 'stop']
Total images: 400
Training complete.
Test Accuracy: 1.00


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt, seaborn as sns

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test,y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(cm,annot=True,fmt="d",xticklabels=labels,yticklabels=labels,cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

print(classification_report(y_test,y_pred,target_names=labels))


In [1]:
import datetime
import tkinter as tk
from tkinter import messagebox


START_HOUR = 18
END_HOUR = 22

def check_time():
    now = datetime.datetime.now().hour
    if START_HOUR <= now < END_HOUR:
        launch_gui()
    else:
        messagebox.showwarning("Access Denied",
                               "Sign Language Detection is only available between 6 PM and 10 PM.")

def launch_gui():
    root = tk.Tk()
    root.title("Sign Language Detection")
    
    
    tk.Label(root, text="Welcome to Sign Language Detection").pack(pady=20)
    tk.Button(root, text="Upload Image", command=lambda: print("Upload logic here")).pack(pady=10)
    tk.Button(root, text="Start Real-Time Video", command=lambda: print("Video logic here")).pack(pady=10)
    
    root.mainloop()

if __name__ == "__main__":
    check_time()


Upload logic here
Upload logic here
Upload logic here
Upload logic here
Upload logic here
Upload logic here
Upload logic here
Upload logic here
Upload logic here
Upload logic here
Upload logic here


In [ ]:
#Requirement text

#opencv
#numpy
#tkinter
tensorflow 
keras
scikit-learn
matplotlib
seaborn


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np

X = np.random.rand(200, 42)
y = np.random.randint(0, 5, 200)
df = pd.DataFrame(np.hstack([X, y.reshape(-1,1)]))
df.to_csv("sign_language.csv", index=False, header=False)

class SignDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file, header=None)
        self.X = self.data.iloc[:, :-1].values.astype(np.float32)
        self.y = self.data.iloc[:, -1].values.astype(np.int64)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class SignNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SignNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

dataset = SignDataset("sign_language.csv")
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
input_size = dataset.X.shape[1]
num_classes = len(np.unique(dataset.y))

model = SignNet(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    for X, y in dataloader:
        outputs = model(X)
        loss = criterion(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

torch.save(model.state_dict(), "task4_sign_model.pt")
print("Model saved as task4_sign_model.pt")


Model saved as task4_sign_model.pt
